In [ ]:
import numpy as np
import pickle
import os
import nibabel as nib
import glob
from skimage import measure
from scipy import ndimage
import matplotlib.pyplot as plt

In [ ]:
def read_mutant_txt(path):
    name_list = []
    fo = open(path)
    for line in fo:
        striped_line = line.strip('\n')
        if striped_line != '':
            name_list.append(striped_line)
    return name_list

def padded_minimum_size(label, min_size=(256,192,160)):
    img_size = np.shape(label)
    x_offset = max(0, round((min_size[0]-img_size[0])/2))
    y_offset = max(0, round((min_size[1]-img_size[1])/2))
    z_offset = max(0, round((min_size[2]-img_size[2])/2))
        
    padded_label=np.zeros((max(img_size[0],min_size[0]),
                           max(img_size[1],min_size[1]),
                           max(img_size[2],min_size[2])),np.uint8)
    
    padded_label[x_offset:x_offset+img_size[0], 
                 y_offset:y_offset+img_size[1], 
                 z_offset:z_offset+img_size[2]] = label
    
    return padded_label

In [ ]:
data_path1 = '/scratch/zq415/grammar_cor/isbi_2020/2seg-localization/data/all_data_bv_body_rotate'
part_body_path2 = './part_body.txt'
nii_save_path = './bv_body_nii'

In [ ]:
files = glob.glob(data_path1 + '/*.nii')
part_body_names = read_mutant_txt(part_body_path2)

In [ ]:
# body_size = []
# img_size = []
# count = 0
# skip_num = 0

# for each_file in files:
#     base_name = os.path.basename(each_file)
#     if base_name[:-4] in part_body_names:
#         skip_num += 1
#         continue
# #     if base_name[:-4] != '20180307_En1_E14p5_M1_Em3abv_body_label':
# #         continue
# #     print(base_name[:-4])
    
#     label = nib.load(each_file)
#     ori_label = np.uint8(label.get_data())
#     label = np.uint8(ori_label > 0.5)
    
#     label_component = measure.label(label)
#     max_component = 1
#     component_num = np.unique(label_component)
#     for current_component in range(2,len(component_num)+1):
#         if np.sum(label_component == current_component) > np.sum(label_component == max_component):
#             max_component = current_component
    
#     body_label = label_component == max_component
    
    
#     body_slice = ndimage.find_objects(body_label)[0]
#     body_size.append([body_slice[0].stop-body_slice[0].start,
#                       body_slice[1].stop-body_slice[1].start,
#                       body_slice[2].stop-body_slice[2].start])
    
#     img_size.append(body_label.shape)
    
#     body_centroid = np.round(ndimage.measurements.center_of_mass(body_label)).astype(int)
    
#     label_cut = ori_label[max(body_centroid[0]-128,0):min(body_centroid[0]+128,img_size[count][0]),
#                           max(body_centroid[1]-96 ,0):min(body_centroid[1]+96 ,img_size[count][0]),
#                           max(body_centroid[2]-80 ,0):min(body_centroid[2]+80 ,img_size[count][0])]
#     if np.sum(label_cut) < 1000:
#         print('something wrong')
    
    
#     padded_label_cut = padded_minimum_size(label_cut)
#     print(count, ': ', body_size[count], img_size[count], 'skip_num: ', skip_num)
#     count += 1
#     print(label_cut.shape, padded_label_cut.shape)
    
#     label_save_path = os.path.join(nii_save_path, base_name)
    
#     padded_label_cut_nib = nib.Nifti1Image(padded_label_cut, np.eye(4))
#     nib.save(padded_label_cut_nib, label_save_path)


In [ ]:
####for checking body size
bvbody_pickle_path = './data/All_raw_bv_body_data.pickle'
bv_size = []
img_size = []
count = 0

with open(bvbody_pickle_path, "rb") as input_file:
    All_raw_bv_body_data = pickle.load(input_file)

unalign_bv = {}

for key in All_raw_bv_body_data:
    bv_label = np.uint8(All_raw_bv_body_data[key] > 1.5)
    img_size.append(bv_label.shape)
#     bv_slice = ndimage.find_objects(bv_label)[0]
#     bv_size.append([bv_slice[0].stop-bv_slice[0].start,
#                     bv_slice[1].stop-bv_slice[1].start,
#                     bv_slice[2].stop-bv_slice[2].start])
    bv_centroid = np.round(ndimage.measurements.center_of_mass(bv_label)).astype(int)
    label_cut = bv_label[max(bv_centroid[0]-64,0):min(bv_centroid[0]+64 ,img_size[count][0]),
                          max(bv_centroid[1]-64 ,0):min(bv_centroid[1]+64 ,img_size[count][0]),
                          max(bv_centroid[2]-64 ,0):min(bv_centroid[2]+64 ,img_size[count][0])]
    
    padded_label_cut = padded_minimum_size(label_cut, min_size=(128,128,128))
    padded_label_cut = padded_label_cut[::2,::2,::2]
    print(count, ': ', padded_label_cut.shape)
    count += 1
    
    unalign_bv[key] = padded_label_cut


In [ ]:
bv_save_path = './data/All_unalign_bv_data_64_64_64_down.pickle'
save_file = open(bv_save_path, 'wb')
pickle.dump(unalign_bv,save_file)
save_file.close()

In [ ]:
####for checking body size
img_size = []
unalign_body = {}
count = 0 

for key in All_raw_bv_body_data:
    body_label = np.uint8(All_raw_bv_body_data[key] > 0.5)
    img_size.append(body_label.shape)

#     bv_slice = ndimage.find_objects(bv_label)[0]
#     bv_size.append([bv_slice[0].stop-bv_slice[0].start,
#                     bv_slice[1].stop-bv_slice[1].start,
#                     bv_slice[2].stop-bv_slice[2].start])
    body_centroid = np.round(ndimage.measurements.center_of_mass(body_label)).astype(int)
    label_cut = body_label[max(body_centroid[0]-128,0):min(body_centroid[0]+128 ,img_size[count][0]),
                          max(body_centroid[1]-128 ,0):min(body_centroid[1]+128 ,img_size[count][0]),
                          max(body_centroid[2]-128 ,0):min(body_centroid[2]+128 ,img_size[count][0])]
    
    padded_label_cut = padded_minimum_size(label_cut, min_size=(256,256,256))
    padded_label_cut = padded_label_cut[::2,::2,::2]
    print(count, ': ', padded_label_cut.shape)
    count += 1
    
    unalign_body[key] = padded_label_cut

    
body_save_path = './data/All_unalign_body_data_128_128_128_down.pickle'
save_file = open(body_save_path, 'wb')
pickle.dump(unalign_body,save_file)
save_file.close()



In [ ]:
####for checking body size
img_size = []
unalign_bv_body = {}
count = 0 

for key in All_raw_bv_body_data:
    bv_body_label = All_raw_bv_body_data[key]
    body_label = np.uint8(bv_body_label > 0.5)
    img_size.append(body_label.shape)

#     bv_slice = ndimage.find_objects(bv_label)[0]
#     bv_size.append([bv_slice[0].stop-bv_slice[0].start,
#                     bv_slice[1].stop-bv_slice[1].start,
#                     bv_slice[2].stop-bv_slice[2].start])
    body_centroid = np.round(ndimage.measurements.center_of_mass(body_label)).astype(int)
    label_cut = bv_body_label[max(body_centroid[0]-128,0):min(body_centroid[0]+128 ,img_size[count][0]),
                          max(body_centroid[1]-128 ,0):min(body_centroid[1]+128 ,img_size[count][0]),
                          max(body_centroid[2]-128 ,0):min(body_centroid[2]+128 ,img_size[count][0])]
    
    padded_label_cut = padded_minimum_size(label_cut, min_size=(256,256,256))
    padded_label_cut = padded_label_cut[::2,::2,::2]
    print(count, ': ', padded_label_cut.shape)
    count += 1
    
    unalign_bv_body[key] = padded_label_cut

    
bv_body_save_path = './data/All_unalign_bv_body_data_128_128_128_down.pickle'
save_file = open(bv_body_save_path, 'wb')
pickle.dump(unalign_bv_body,save_file)
save_file.close()


In [ ]:
plt.imshow(unalign_bv_body[key][:,:,50])

In [ ]:
count=0
all_data={}
miss_img1=0
miss_img2=0
miss_img3=0
miss_name=[]
bv_ratio = []
size_min = 1000
size_max = 0

for folder1 in folder_list1:
    data_path2=os.path.join(data_path1,folder1)
    folder_list2=os.listdir(data_path2)
    folder_list2=[x for x in folder_list2 if not x.startswith('.')]
    for folder2 in folder_list2:
        data_path3=os.path.join(data_path2,folder2)
        folder_list3=os.listdir(data_path3)
        folder_list3=[x for x in folder_list3 if (not x.endswith('labels.nii') and not x.startswith('.')
                                                 and not x.endswith('filtered.nii'))]
        for folder3 in folder_list3:
            data_path4=os.path.join(data_path3,folder3)
            #filtered_path=data_path4[:-4] + '.nii'#'_filtered.nii'
            label_path = os.path.join(data_path4[:-4] + '_BV_labels.nii')
            
            img = nib.load(data_path4)
            img = np.float32(img.get_data())

#             filtered_img = nib.load(filtered_path)
#             filtered_img = np.float32(filtered_img.get_data())
            label = nib.load(label_path)
            label = np.uint8(label.get_data())
#             if np.shape(img) != np.shape(label) or np.shape(img) != np.shape(filtered_img):
#                 miss_img1+=1
#                 miss_name.append(data_path4)
#                 continue
#             if np.shape(img)[0] < 80 or np.shape(img)[1] < 80 or np.shape(img)[2] < 80:
#                 miss_img2+=1
#                 miss_name.append(data_path4)
#                 continue
#             if np.sum(label) <6000:
#                 miss_img3+=1
#                 miss_name.append(data_path4)
#                 continue

            all_data[count]=(data_path4,label_path)
            count+=1
            print(count,'  ',np.shape(img),'  ',np.shape(label),'  ',miss_img1, '  ',
                  miss_img2,'  ', miss_img3,'  ', data_path4)
            #bv_ratio.append(np.sum(label)/(np.prod(np.shape(img))+0.00001))
            #print('bv ratio: {}'.format(bv_ratio[count-1]))
#             if min(np.shape(img)) < size_min:
#                 size_min = min(np.shape(img))
#             if max(np.shape(img)) > size_max:
#                 size_max = max(np.shape(img))

In [ ]:
save_name = 'All_2test_data_path.pickle'
save_file = open(os.path.join(os.getcwd(),'data',save_name),'wb')
pickle.dump(all_data,save_file)
save_file.close()

In [ ]:
print('min_size: {}, max_size: {}'.format(size_min, size_max))

In [ ]:
file_path = os.path.join(os.getcwd(),'data',save_name)
with open(file_path,'rb') as f:
    data_dic = pickle.load(f)
f.close()

In [ ]:
print(data_dic)

In [ ]:
avg_bv_ratio = 0
for i in range(len(bv_ratio)):
    avg_bv_ratio += bv_ratio[i]
avg_bv_ratio /= len(bv_ratio)
print(avg_bv_ratio)

In [ ]:
np.unique(img)